## Data Preprocessing

### Use Spark to get data

In [2]:
%help

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 



# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %timeout            Int           The number of minutes after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session.
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0, 3.0 and 4.0. 
                                      Default: 2.0.
    %reconnect          String        Specify a live session ID to switch/reconnect to the sessions.
----

## Selecting Session Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %session_type       String        Specify a session_type to be used. Supported values: streaming and etl.
----

## Glue Config Magic 
*(common across all session types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
    %%tags        Dictionary          Specify a json-formatted dictionary consisting of tags to use in the session.
    
    %%assume_role Dictionary, String  Specify a json-formatted dictionary or an IAM role ARN string to create a session 
                                      for cross account access.
                                      E.g. {valid arn}
                                      %%assume_role 
                                      'arn:aws:iam::XXXXXXXXXXXX:role/AWSGlueServiceRole' 
                                      E.g. {credentials}
                                      %%assume_role
                                      {
                                            "aws_access_key_id" : "XXXXXXXXXXXX",
                                            "aws_secret_access_key" : "XXXXXXXXXXXX",
                                            "aws_session_token" : "XXXXXXXXXXXX"
                                       }
----

                                      
## Magic for Spark Sessions (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
    %matplot      Matplotlib figure   Visualize your data using the matplotlib library.
                                      E.g. 
                                      import matplotlib.pyplot as plt
                                      # Set X-axis and Y-axis values
                                      x = [5, 2, 8, 4, 9]
                                      y = [10, 4, 8, 5, 2]
                                      # Create a bar chart 
                                      plt.bar(x, y) 
                                      # Show the plot
                                      %matplot plt    
    %plotly            Plotly figure  Visualize your data using the plotly library.
                                      E.g.
                                      import plotly.express as px
                                      #Create a graphical figure
                                      fig = px.line(x=["a","b","c"], y=[1,3,2], title="sample figure")
                                      #Show the figure
                                      %plotly fig

  
                
----



### Setup the session

In [1]:
import boto3
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window

spark = SparkSession.builder \
    .appName("CryptoDataProcessing") \
    .getOrCreate()

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Trying to create a Glue session for the kernel.
Session Type: etl
Session ID: 5c29d583-1396-4994-90fc-74ad9ce23eaa
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 5c29d583-1396-4994-90fc-74ad9ce23eaa to get into ready status...
Session 5c29d583-1396-4994-90fc-74ad9ce23eaa has been created.



In [2]:
%session_id 

Current active Session ID: 5c29d583-1396-4994-90fc-74ad9ce23eaa


In [27]:
# Define file path to access bucket
bucket_name = "crypto-marketdata-marketdatabucket-m13jkiz8o4p7"
period = "1h"
year="2017"
btc_file_path = "s3://{}/marketdata/crypto/BTCUSDT-{}-{}-*.snappy.parquet".format(bucket_name, period, year)

In [28]:
# Load the data
btc_data = spark.read.parquet(btc_file_path)

In [29]:
btc_data.head(1)

[Row(open_time=datetime.datetime(2017, 12, 1, 1, 0), open=9685.02, high=9720.0, low=9550.0, close=9635.0, vol=279.578526, close_time=datetime.datetime(2017, 12, 1, 1, 59, 59, 999000), quote_asset_vol=2690971.25173209, number_of_trades=1488, taker_base_asset_vol=137.232568, taker_quote_asset_vol=1321250.32929446, ignore=34409.30772883, ticker='BTCUSDT')]


### Extra data

In [17]:
def calculate_sma(df, window_size):
    return df.withColumn("SMA_{}".format(window_size), F.avg("close").over(Window.orderBy("close_time").rowsBetween(-window_size + 1, 0)))

In [31]:
# Calculate Simply Moving Average (SMA) for both BTC and ETH for 10 days(240 hr) and 20 days(480 hr)

btc_data = calculate_sma(btc_data, 240)
btc_data = calculate_sma(btc_data, 480)

In [32]:
btc_data.head(1)

[Row(open_time=datetime.datetime(2017, 8, 17, 5, 0), open=4308.83, high=4328.69, low=4291.37, close=4315.32, vol=23.234916, close_time=datetime.datetime(2017, 8, 17, 5, 59, 59, 999000), quote_asset_vol=100304.82356749, number_of_trades=102, taker_base_asset_vol=21.448071, taker_quote_asset_vol=92608.27972836, ignore=8039.26240152, ticker='BTCUSDT', SMA_240=4315.32, SMA_480=4315.32)]


In [34]:
def calculate_rsi(df, window_size):
    columns = ["close_time", "close"]
    rsi = df.select(columns)
    rsi = rsi.withColumn("delta", F.col("close") - F.lag("close").over(Window.orderBy("close_time")))
    rsi = rsi.withColumn("gain", F.when(F.col("delta") > 0, F.col("delta")).otherwise(0))
    rsi = rsi.withColumn("loss", F.when(F.col("delta") < 0, -F.col("delta")).otherwise(0))

    window = Window.orderBy("close_time").rowsBetween(-window_size + 1, 0)
    rsi = rsi.withColumn("avg_gain", F.avg("gain").over(window))
    rsi = rsi.withColumn("avg_loss", F.avg("loss").over(window))

    rsi = rsi.withColumn("rs", F.col("avg_gain") / F.col("avg_loss"))
    rsi = rsi.withColumn("RSI", 100 - (100 / (1 + F.col("rs"))))

    return df.join(rsi.select(["RSI", "close_time"]), on="close_time", how="left")

In [35]:
# Calculate Relative Strength Index (RSI) for both BTC and ETH Over 336 hr (14 days)
btc_data = calculate_rsi(btc_data, 336)

In [36]:
btc_data.head(5)

[Row(close_time=datetime.datetime(2017, 8, 17, 5, 59, 59, 999000), open_time=datetime.datetime(2017, 8, 17, 5, 0), open=4308.83, high=4328.69, low=4291.37, close=4315.32, vol=23.234916, quote_asset_vol=100304.82356749, number_of_trades=102, taker_base_asset_vol=21.448071, taker_quote_asset_vol=92608.27972836, ignore=8039.26240152, ticker='BTCUSDT', SMA_240=4315.32, SMA_480=4315.32, RSI=None), Row(close_time=datetime.datetime(2017, 8, 17, 6, 59, 59, 999000), open_time=datetime.datetime(2017, 8, 17, 6, 0), open=4330.29, high=4345.45, low=4309.37, close=4324.35, vol=7.229691, quote_asset_vol=31282.31266989, number_of_trades=36, taker_base_asset_vol=4.802861, taker_quote_asset_vol=20795.31722405, ignore=8041.76049845, ticker='BTCUSDT', SMA_240=4319.835, SMA_480=4319.835, RSI=None), Row(close_time=datetime.datetime(2017, 8, 17, 7, 59, 59, 999000), open_time=datetime.datetime(2017, 8, 17, 7, 0), open=4316.62, high=4349.99, low=4287.41, close=4349.99, vol=4.443249, quote_asset_vol=19241.05829

In [21]:
def calculate_bollinger_bands(df, window_size, k):
    window_spec = Window.orderBy("close_time").rowsBetween(-window_size + 1, 0)

    # find Middle Bollinger Bands
    df = df.withColumn("Middle_Band", F.avg("close").over(window_spec))

    # Find standard deviation
    df = df.withColumn("STD", F.stddev("close").over(window_spec))

    # Find the Upper and Lower Bollinger Bands
    df = df.withColumn("Upper_Band", F.col("Middle_Band") + (F.col("STD") * k))
    df = df.withColumn("Lower_Band", F.col("Middle_Band") - (F.col("STD") * k))

    return df

In [38]:
# Calculate Bollinger Bands for both BTC and ETH with period = 20 days
btc_data = calculate_bollinger_bands(btc_data, 480, 2)

In [39]:
btc_data.head(1)

[Row(close_time=datetime.datetime(2017, 8, 17, 5, 59, 59, 999000), open_time=datetime.datetime(2017, 8, 17, 5, 0), open=4308.83, high=4328.69, low=4291.37, close=4315.32, vol=23.234916, quote_asset_vol=100304.82356749, number_of_trades=102, taker_base_asset_vol=21.448071, taker_quote_asset_vol=92608.27972836, ignore=8039.26240152, ticker='BTCUSDT', SMA_240=4315.32, SMA_480=4315.32, RSI=None, Middle_Band=4315.32, STD=None, Upper_Band=None, Lower_Band=None)]


### Upload the processed data

In [41]:
btc_output_path = f"s3://{bucket_name}/marketdata/processed_crypto/BTCUSDT-{year}-processed.parquet"

In [42]:
btc_data.write.mode("overwrite").parquet(btc_output_path)

In [6]:
!aws s3 ls s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/processed_crypto --recursive

2024-11-29 17:07:01     427631 marketdata/processed_crypto/BTCUSDT-2017-processed.parquet/part-00000-deeebcd2-d2be-4b56-b54b-d5305c2e91e6-c000.snappy.parquet
2024-11-29 17:05:38    1077173 marketdata/processed_crypto/BTCUSDT-2018-processed.parquet/part-00000-a4adf4c6-3342-42d9-a529-0af46568861b-c000.snappy.parquet
2024-11-29 16:55:20    1077146 marketdata/processed_crypto/BTCUSDT-2019-processed.parquet/part-00000-6b9c2612-ca5d-49fd-8609-9d6c3045a9fb-c000.snappy.parquet
2024-11-28 08:02:47    1087753 marketdata/processed_crypto/BTCUSDT-2020-processed.parquet/part-00000-7f7ad95e-45e7-4289-b615-98ccbcc577c0-c000.snappy.parquet
2024-11-28 08:02:15    1095992 marketdata/processed_crypto/BTCUSDT-2021-processed.parquet/part-00000-26003231-5fbf-4e46-b8d4-a95c125e8f34-c000.snappy.parquet
2024-11-28 08:01:27    1095197 marketdata/processed_crypto/BTCUSDT-2022-processed.parquet/part-00000-3e7c82d7-3002-456c-9f3e-e7022ce8bffa-c000.snappy.parquet
2024-11-28 08:00:24    1091682 marketdata/processed_

In [8]:
%stop_session 

Stopping session: 5c29d583-1396-4994-90fc-74ad9ce23eaa
Stopped session.
